In [32]:
from py2neo import Graph
from py2neo import Node, Relationship
import json
import pandas as pd
import sys

In [33]:
graph = Graph('http://localhost:7474', password='1234')

In [62]:
keyword = 'imran khan'

In [63]:
entities = ['imran khan', 'nawaz sharif', 'bilawal bhutto', 'asif zardari',
            'arif alvi', 'maryam nawaz', 'asif khosa', 'fawad chaudhry', 'fazal ur rehman',
            'shehbaz sharif', 'qamar bajwa', 'altaf hussain', 'pervez musharraf',
            'mustafa kamal', 'siraj ul haq', 'sheikh rasheed', 'pervez khattak', 'asad umar',
            'murad ali shah', 'aitzaz ahsan', 'asif ghafoor', 'PTI', 'PMLN', 'PPP', 'JUI', 'MQM']

In [70]:
file = 'model/summary_' + keyword + '_10_topics.json'
twitter_file = 'model/topic-tweets_' + keyword + '_10_topics.json'
youtube_file = 'model/topic-videos_' + keyword + '_10_topics.json'

with open(twitter_file) as json_file:
    tweet_data = json.load(json_file)

with open(youtube_file) as json_file:
    yt_data = json.load(json_file)


with open(file) as json_file:
    data = json.load(json_file)

In [71]:
import random
tweet_data = random.sample(tweet_data, 50)
yt_data = random.sample(yt_data, 30)

In [72]:
### Creating summaries

for i in range(0, len(data)):

    summary = Node("Story", Topic=data[i]['topic']
                         , Summary=data[i]['Summary']
                         , Sentiment=data[i]['Sentiment']
                         , Date=data[i]['Date'])

    graph.create(summary)
    
    ###
    
    for entity in entities:
        if entity in data[i]['Summary'].lower():
            node = graph.evaluate('MATCH(e:Entity {name: "' + entity + '"}) RETURN e')
            if not node:
                node = graph.evaluate('CREATE(e:Entity {name: "' + entity + '"}) RETURN e')
                
            r = Relationship(node, "HAS_STORY", summary)
            graph.create(r)
            
    ###
    

    # Creating tweets and videos belonging to that summary

    for j in range(0, len(tweet_data)):
        if tweet_data[j]['Topic'] == i:
            
            tweet_type = tweet_data[j]['Type']                
                
            tweet = Node(tweet_type, Created_time = tweet_data[j]['Created_time']
                                , URL = tweet_data[j]['URL']
                                , User_name = tweet_data[j]['User_name']
                                , Twitter_handle = tweet_data[j]['Twitter_handle']
                                , Description = tweet_data[j]['Description']
                                , Retweet_count = tweet_data[j]['Retweet_count']
                                , Favorite_count = tweet_data[j]['Favorite_count']
                                , Sentiment = tweet_data[j]['Sentiment']
                                , Topic = tweet_data[j]['Topic'])
            
            graph.create(tweet)

            r = Relationship(summary, "HAS_TWEET", tweet)

            graph.create(r)
    
    
    for j in range(0, len(yt_data)):
        if yt_data[j]['Topic'] == i:
            video = Node("Video", Published_date = yt_data[j]['Published_date']
                                , Title = yt_data[j]['Title']
                                , URL = yt_data[j]['URL']
                                , Channel_id = yt_data[j]['Channel_id']
                                , Topic = yt_data[j]['Topic'])

            graph.create(video)

            r = Relationship(summary, "HAS_VIDEO", video)

            graph.create(r)
    